In [16]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

directory = './1_CancerGov_QA/'

qa_list = []

for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        file_path = os.path.join(directory, filename)
        
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        for qapair in root.find('QAPairs'):
            question = qapair.find('Question').text.strip()
            answer = qapair.find('Answer').text.strip()
            qa_list.append({'Question': question, 'Answer': answer})

df = pd.DataFrame(qa_list)

print(df.head())

                                   Question  \
0             What is (are) Penile Cancer ?   
1       Who is at risk for Penile Cancer? ?   
2  What are the symptoms of Penile Cancer ?   
3           How to diagnose Penile Cancer ?   
4   What is the outlook for Penile Cancer ?   

                                              Answer  
0  Key Points\n                    - Penile cance...  
1  Human papillomavirus infection may increase th...  
2  Signs of penile cancer include sores, discharg...  
3  Tests that examine the penis are used to detec...  
4  Certain factors affect prognosis (chance of re...  


In [17]:
import xml.etree.ElementTree as ET
import pandas as pd
import random

qa_pairs = []
for idx, i in enumerate(df["Question"]):
    question = i
    answer = df["Answer"][idx]
    qa_pairs.append((question, answer))

prompts = []

for i, (question, correct_answer) in enumerate(qa_pairs):
    other_answers = [answer for _, answer in qa_pairs if answer != correct_answer]
    random_answers = random.sample(other_answers, 3)

    prompt = f'Example 1: "{random_answers[0]}"\n\n'
    prompt += f'Example 2: "{random_answers[1]}"\n\n'
    prompt += f'Example 3: "{random_answers[2]}"\n\n'
    prompt += f'Please read the example answer below for other questions, and then answer the question briefly in a similar style (using lists if you have multiple points after a brief summary): "{question}". You need to use "-" to index your list and write the answer in a code environment. Do not include "*".'

    prompts.append(prompt)

df_prompts = pd.DataFrame(prompts, columns=['Prompt'])
print(df_prompts.head())

                                              Prompt
0  Example 1: "New types of treatment are being t...
1  Example 1: "Key Points\n                    - ...
2  Example 1: "Signs of colon cancer include bloo...
3  Example 1: "Key Points\n                    - ...
4  Example 1: "Key Points\n                    - ...


In [18]:
from openai import OpenAI
client = OpenAI(api_key="your_api")

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": df_prompts['Prompt'][0]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response)

ChatCompletion(id='chatcmpl-9FQf5o0FXLvMSXGtmM88exWlcI0pS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```\n- Penile cancer is a rare type of cancer that occurs on the skin or tissues of the penis.\n```', role='assistant', function_call=None, tool_calls=None))], created=1713464435, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=24, prompt_tokens=6747, total_tokens=6771))


In [19]:
def remove_md(text):
    text = text.replace("```markdown\n", "")
    text = text.replace("\n```", "")
    text = text.replace("```", "")
    return text

print(remove_md(response.choices[0].message.content))


- Penile cancer is a rare type of cancer that occurs on the skin or tissues of the penis.


In [20]:
from openai import OpenAI
from tqdm import tqdm
client = OpenAI(api_key="your_api")

def remove_md(text):
    text = text.replace("```markdown\n", "")
    text = text.replace("\n```", "")
    return text

def gpt_response(prompt):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
        "role": "user",
        "content": prompt
    }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return remove_md(response.choices[0].message.content)

def save_checkpoint(index):
    with open("checkpoint.txt", "w") as f:
        f.write(str(index))

def load_checkpoint():
    try:
        with open("checkpoint.txt", "r") as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return 0

df.to_csv("data.csv")
df_prompts.to_csv("prompts.csv")

In [21]:
df = pd.read_csv("data.csv")
df_prompts = pd.read_csv("prompts.csv")

start_index = load_checkpoint()
answers = [None] * len(df)  # Pre-allocate list for answers

for idx in tqdm(range(start_index, len(df["Question"]))):
    prompt = df_prompts['Prompt'][idx]
    try:
        answer = gpt_response(prompt)
    except Exception as e:
        print(f"Error processing prompt at index {idx}: {e}")
        answer = "Error"
    if answer != "Error":
        answers[idx] = answer
        df.at[idx, "AI Answers"] = answer
        save_checkpoint(idx + 1)

df.to_csv("updated_answers.csv", index=False)

100%|██████████| 729/729 [31:37<00:00,  2.60s/it]


In [22]:
df.to_csv("1_CancerGov_QA.csv")

In [23]:
# print(df["AI Answers"][60])
# print(df["Question"][60])
print(df_prompts["Prompt"][60])

Example 1: "Certain factors affect prognosis (chance of recovery) and treatment options. The prognosis (chance of recovery) and treatment options depend on the following:         - The stage of the cancer (whether the cancer is in the inner lining of the colon only or has spread through the colon wall, or has spread to lymph nodes or other places in the body).    - Whether the cancer has blocked or made a hole in the colon.    - Whether there are any cancer cells left after surgery.    - Whether the cancer has recurred.     - The patients general health.        The prognosis also depends on the blood levels of carcinoembryonic antigen (CEA) before treatment begins. CEA is a substance in the blood that may be increased when cancer is present."

Example 2: "Signs and symptoms of neuroblastoma include bone pain and a lump in the abdomen, neck, or chest.The most common signs and symptoms of neuroblastoma are caused by the tumor pressing on nearby tissues as it grows or by cancer spreading 